In [1]:
%matplotlib inline
from __future__ import print_function
import hera_cal.redcal as om
import hera_cal.omni
import omnical.calib
import numpy as np
import time
import uvtools
import pylab as plt
from scipy.stats import kde, chi2

In [ ]:
def build_hex_array(hexNum, sep=14.7):
    antpos, i = {}, 0
    for row in range(hexNum - 1, -(hexNum), -1):
        for col in range(2 * hexNum - abs(row) - 1):
            xPos = ((-(2 * hexNum - abs(row)) + 2) / 2.0 + col) * sep
            yPos = row * sep * 3**.5 / 2
            antpos[i] = np.array([xPos, yPos, 0])
            i += 1
    return antpos

In [ ]:
np.random.seed(0)
SHAPE = (20,1024)
NANTS = 37
NOISE = 1e-3

Nants = 19

ants = np.loadtxt('antenna_positions_%d.dat'%Nants)
idxs = np.arange(Nants)
antpos = {}
for k,v in zip(idxs,ants):
    antpos[k] = v
    
reds = om.get_reds(antpos, pols=['xx'], pol_mode='1pol')

info = om.RedundantCalibrator(reds)

gains, true_vis, d = om.sim_red_data(reds, shape=SHAPE, gain_scatter=.01)
d = {key: value.astype(np.complex64) for key,value in d.items()}
d_nos = {key: value + NOISE * om.noise(value.shape) for key,value in d.items()}
d_nos = {key: value.astype(np.complex64) for key,value in d_nos.items()}

w = dict([(k, np.float32(1.)) for k in d.keys()])
sol0 = dict([(k, np.ones_like(v)) for k, v in gains.items()])
sol0.update(info.compute_ubls(d, sol0))
sol0 = {k:v.astype(np.complex64) for k,v in sol0.items()}

In [ ]:
def gen_chisq(data, sol, noise):
    d_mdl = {}
    for bls in reds:
        if len(bls) <= 1: continue
        ubl = sol[bls[0]]
    for bl in bls:
        d_mdl[(bl[0],bl[1],'xx')] = \
        sol[(bl[0],'x')] * sol[(bl[1],'x')].conj() * ubl
    res = [data[k] - v for k,v in d_mdl.items()]
    chisq = np.sum(np.abs(res)**2 / noise**2, axis=0) / (len(res) - len(sol))
    return chisq

In [ ]:
def plot_gain_residuals(sol, true_gains, lim, nbins=20):
    sol_degen = info.remove_degen(sol, degen_sol=true_gains)
    keys = [k for k in sol.keys() if len(k) == 2]
    fig,(ax,ax2) = plt.subplots(1,2,sharey=True,figsize=(8,4))
    
    for k in keys:
        res = sol_degen[k] - true_gains[k]
        res = res.flatten()
        res = res[np.where(np.isfinite(res))]
 
        _ = ax.plot(res.real, res.imag, '.', alpha=.2)
        res = np.array([res.real, res.imag])
        x,y = res
        k = kde.gaussian_kde(res)
        xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
        zi = k(np.vstack([xi.flatten(), yi.flatten()]))
        
        ax2.grid()
        _ = ax2.contour(xi, yi, zi.reshape(xi.shape), alpha=.5)
        
    ax2.set_xlabel('$\Delta$g, real')
    ax2.set_xlim(-lim,lim)
    #ax2.setp(ax2.get_yticklabels(), visible=False)
    ax2.grid(b=True, which='major', color='k', linestyle='-')
    
    ax.set_xlabel('$\Delta$g, real')
    ax.set_ylabel('$\Delta$g, imag')
    ax.set_xlim(-lim,lim); plt.ylim(-lim,lim)
    ax.grid(b=True, which='major', color='k', linestyle='-')

In [ ]:
# LOGCAL
t0 = time.time()
sol_logcal = info.logcal(d, wgts=w)
print('LOGCAL: %4.1f s' % (time.time() - t0),
      sol_logcal.values()[0].dtype)

t0 = time.time()
sol_nos_logcal = info.logcal(d_nos, wgts=w)
print('LOGCAL: %4.1f s' % (time.time() - t0),
      sol_nos_logcal.values()[0].dtype)

In [ ]:
plot_gain_residuals(sol_nos_logcal, gains, 0.02)

In [ ]:
# OMNICAL
t0 = time.time()
meta_omnical, sol_omnical = info.omnical(d, sol0, 
                                         gain=.5, maxiter=500, check_after=30, check_every=6)
print('OMNICAL: %4.1f s' % (time.time() - t0), 
      sol_omnical.values()[0].dtype)

t0 = time.time()
meta_nos_omnical, sol_nos_omnical = info.omnical(d_nos, sol0, 
                                                 gain=.5, maxiter=500, check_after=30, check_every=6)
print('OMNICAL: %4.1f s' % (time.time() - t0), 
      sol_nos_omnical.values()[0].dtype)

In [ ]:
np.random.seed(0)
SHAPE = (10,2048)
NOISE = 1e-3

for Nants in [37, 128, 243, 350]:

    ants = np.loadtxt('antenna_positions_%d.dat'%Nants)
    idxs = np.arange(Nants)
    antpos = {}
    for k,v in zip(idxs,ants):
        antpos[k] = v

    reds = om.get_reds(antpos, pols=['xx'], pol_mode='1pol')

    info = om.RedundantCalibrator(reds)

    gains, true_vis, d = om.sim_red_data(reds, shape=SHAPE, gain_scatter=.01)
    d = {key: value.astype(np.complex64) for key,value in d.items()}
    d_nos = {key: value + NOISE * om.noise(value.shape) for key,value in d.items()}
    d_nos = {key: value.astype(np.complex64) for key,value in d_nos.items()}

    w = dict([(k, np.float32(1.)) for k in d.keys()])
    sol0 = dict([(k, np.ones_like(v)) for k, v in gains.items()])
    sol0.update(info.compute_ubls(d, sol0))
    sol0 = {k:v.astype(np.complex64) for k,v in sol0.items()}
    
    print('NANTS: %d\n\n'%Nants)
    
    t0 = time.time()
    meta_omnical, sol_omnical = info.omnical(d, sol0, 
                                             gain=.5, maxiter=500, check_after=30, check_every=6)
    print('OMNICAL no noise: %4.1f s' % (time.time() - t0), 
          sol_omnical.values()[0].dtype)

    t0 = time.time()
    meta_nos_omnical, sol_nos_omnical = info.omnical(d_nos, sol0, 
                                                     gain=.5, maxiter=500, check_after=30, check_every=6)
    print('OMNICAL with noise: %4.1f s' % (time.time() - t0), 
          sol_nos_omnical.values()[0].dtype)

NANTS: 37


OMNICAL no noise: 27.6 s complex64
OMNICAL with noise: 12.1 s complex64
NANTS: 128


